In [1]:
import argparse
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from importlib import import_module

import torch
from dataset import CustomDataLoader, collate_fn, test_transform
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F

In [2]:
output_dir = './output'

# load sample_submission.csv
submission = pd.read_csv('../baseline_code/submission/sample_submission.csv', index_col=None)

In [3]:
print("Start prediction..")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

test_dataset = CustomDataLoader(data_dir='/opt/ml/segmentation/input/data/test.json', mode='test', transform=test_transform)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=8,
    num_workers=4,
    shuffle=False,
    pin_memory=use_cuda,
    collate_fn=collate_fn
)

Start prediction..
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
import ttach as tta
transforms = tta.Compose(
[
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 180]),
    # tta.Resize(sizes=(512,512)),
    # tta.Scale(scales=[1, 2, 4]),
    tta.Multiply(factors=[0.9, 1, 1.1]),      
])

In [5]:
    models_name = ['DeepLabV3_Plus', 'TransUnet']
    ckpt_list = ['/opt/ml/segmentation/semantic-segmentation-level2-cv-06/runs/deeplab_v3_b7_best.pt',
                '/opt/ml/segmentation/semantic-segmentation-level2-cv-06/runs/transunet_SGD_0.61_0.63.pt']
    
    # model 1 가중치 불러오기
    model_module = getattr(import_module("models.model"), models_name[0])
    model1 = model_module(num_classes=11, pretrained=True)
    checkpoint = torch.load(ckpt_list[0], map_location=device)
    state_dict = checkpoint.state_dict()
    model1.load_state_dict(state_dict)
    model1.to(device)

    # model 2 가중치 불러오기
    model_module = getattr(import_module("models.model"), models_name[1])
    model2 = model_module(num_classes=11, pretrained=True)
    checkpoint = torch.load(ckpt_list[1], map_location=device)
    state_dict = checkpoint.state_dict()
    model2.load_state_dict(state_dict)
    model2.to(device)

    # wrapper로 TTA augmentation 적용
    tta_model1 = tta.SegmentationTTAWrapper(model1, transforms)
    tta_model2 = tta.SegmentationTTAWrapper(model2, transforms)
    
    # 각 모델 eval로 type 변경
    model1.eval()
    tta_model1.eval()
    model2.eval()
    tta_model2.eval()

load_pretrained: grid-size from 24 to 32


SegmentationTTAWrapper(
  (model): TransUnet(
    (backbone): VisionTransformer(
      (transformer): Transformer(
        (embeddings): Embeddings(
          (hybrid_model): ResNetV2(
            (root): Sequential(
              (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
              (gn): GroupNorm(32, 64, eps=1e-06, affine=True)
              (relu): ReLU(inplace=True)
            )
            (body): Sequential(
              (block1): Sequential(
                (unit1): PreActBottleneck(
                  (gn1): GroupNorm(32, 64, eps=1e-06, affine=True)
                  (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (gn2): GroupNorm(32, 64, eps=1e-06, affine=True)
                  (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (gn3): GroupNorm(32, 256, eps=1e-06, affine=True)
                  (conv3): StdConv2d(64, 256, kerne

In [6]:
size = 256
transform = A.Compose([A.Resize(size, size)])

file_name_list = []
preds_array = np.empty((0, size*size), dtype=np.compat.long)

In [7]:
 with torch.no_grad():
    for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
        # inference (512 x 512)
        outs = 0
        for model_name in models_name:
            if model_name in ('FCNRes50', 'FCNRes101', 'DeepLabV3_Res50', 'DeepLabV3_Res101'):
                # output size : ([8, 11, 512, 512])
                outs += model1(torch.stack(imgs).to(device))['out']
                outs += tta_model1(torch.stack(imgs).to(device))['out']
                # print('if : ', model_name)
            else:
                # 원하는 모델을 더해주고 더해준 모델의 개수만큼 나눠준다.
                images = torch.stack(imgs).to(device)
                
                outs += model1(images)
                outs += tta_model1(images)

                # transunet input size (1024,1024)로 변경하여 넣어주기
                image_ = []
                for i in imgs:
                    i = i.squeeze()
                    i = torch.transpose(i, 0,2)
                    i_ = cv2.resize(np.float32(i), (1024,1024), interpolation = cv2.INTER_CUBIC)
                    i_ = torch.transpose(torch.tensor(i_), 0,2)
                    image_.append(i_)
                transunet_image = torch.stack(image_).to(device)

                outs += model2(transunet_image)
                outs += tta_model2(transunet_image)

                # print('else : ', model_name)
                # outs = (1/4)*(model1(images)+tta_model1(images)+model2(images)+tta_model2(images))   
            outs = (1/4)*outs
 
        oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy() # (8, 512, 512)

        # resize (256 x 256)
        temp_mask = []
        for img, mask in zip(np.stack(imgs), oms):
            transformed = transform(image=img, mask=mask)
            mask = transformed['mask']
            temp_mask.append(mask)


        oms = np.array(temp_mask)
        oms = oms.reshape([oms.shape[0], size*size]).astype(int)
        preds_array = np.vstack((preds_array, oms))

        file_name_list.append([i['file_name'] for i in image_infos])

print("End prediction!")
file_names = [y for x in file_name_list for y in x]

  0%|          | 0/103 [00:00<?, ?it/s]/opt/conda/envs/segmentation/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/opt/conda/envs/segmentation/lib/python3.7/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
100%|██████████| 103/103 [42:48<00:00, 20.0

End prediction!


In [ ]:
import pickle

# save
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# load
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [8]:
# write PredictionString
for file_name, string in zip(file_names, preds_array):
    submission = submission.append({"image_id": file_name, "PredictionString": ' '.join(str(e) for e in string.tolist())},
                                    ignore_index=True)

# save submission.csv
submission.to_csv(output_dir+'/submission.csv', index=False)